<img src="./images/react.png" width=700px>

NB This talk is less about creating Agentic Apps as it is to demystify and simplify what AI Agents are. In this regard, this example may seem involved on seeing it the fist time. Look for where we have Day To Day Python and where we have AI, so that you may see that AI Agents are 'just' Python with LLM calls.


This example shows planning using ReAct - Reason and Act.

Essentially, we pass the output of each step to the next request to the LLM, like we did in reflection but adding in tool calling.

PLANNING is equivalent to REFLECTION and TOOL CALLING

We continue to do this until we get to the end as specified in the prompt, which we will see later in the code.

By giving examples, we are using Multi-Shot Prompting. If we gave just one example then this is called Single Shot Prompting, and no examples is Zero Shot Prompting.

We may go through it iteratively in this notebook but the next file `20_planning_agent_w_loop.py` has this as a Class containing a loop and form a high level view is easy to grasp.


NB This example may need to be run through a number of times to get it to sink in so just focus on the high level view.


We will ask the agent to get the total price including VAT for a product.

It will have two tools:

1. get_product_price(product)
2. calculate_total(price * 1.2 VAT)

We will ask it to get the total price for say a laptop. It will need to REASON how to do this, take ACTION, get an OBSERVATION and repeat this loop until it gets an ANSWER.

This script has been detailed in the SYSTEM PROMPT.

It is essentially a loop passing back in the former output until it thinks it has an ANSWER.

*Let's not get focused on the code implementation but rather see the overall flow pattern. We can digest it more fully offline*

In [1]:
import os
from openai import OpenAI
from dotenv import load_dotenv

In [2]:
def get_llm_client(llm_choice):
    if llm_choice == "GROQ":
        client = OpenAI(
            base_url="https://api.groq.com/openai/v1",
            api_key=os.environ.get("GROQ_API_KEY"),
        )
        return client
    elif llm_choice == "OPENAI":
        load_dotenv()  # load environment variables from .env fil
        client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
        return client
    else:
        raise ValueError("Invalid LLM choice. Please choose 'GROQ' or 'OPENAI'.")

In [3]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

LLM_CHOICE = "OPENAI"
# breaks with Groq as the output is not in the right format. This shows the need for structured output whcih we will see in 10_groq_strucutred _output.ipynb
# LLM_CHOICE = "GROQ"

if OPENAI_API_KEY:
    print(f"OPENAI_API_KEY exists and begins {OPENAI_API_KEY[:14]}...")
else:
    print("OPENAI_API_KEY not set")

if GROQ_API_KEY:
    print(f"GROQ_API_KEY exists and begins {GROQ_API_KEY[:14]}...")
else:
    print("GROQ_API_KEY not set")


client = get_llm_client(LLM_CHOICE)
if LLM_CHOICE == "GROQ":
    MODEL = "llama-3.3-70b-versatile"
else:
    MODEL = "gpt-4o-mini"

print(f"LLM_CHOICE: {LLM_CHOICE} - MODEL: {MODEL}")

OPENAI_API_KEY exists and begins sk-proj-1WUVgv...
GROQ_API_KEY exists and begins gsk_11hFN1EMfj...
LLM_CHOICE: OPENAI - MODEL: gpt-4o-mini


In [4]:
# We can make a class to show the ReAct (Reason - Act) pattern
class PlanningAgent:
    def __init__(self, client, model=MODEL, system: str = "") -> None:
        self.client = client
        self.system = system
        self.messages: list = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message=""):
        if message:
            self.messages.append({"role": "user", "content": message})
            result = self.execute()
            self.messages.append({"role": "assistant", "content": result})
            return result
        else:
            print("NO MESSAGE")

    def execute(self):
        completion = client.chat.completions.create(model=MODEL, messages=self.messages)
        return completion.choices[0].message.content

In [5]:
system_prompt = """
You run in a loop of THOUGHT, ACTION, OBSERVATION.

You have two tools available for your ACTIONS - calculate_total and get_product_price so that you can get the total price of an item requested by the user.

# 1. calculate_total:

if amount = 200
then calculate_total(amount)
return amount * 1.2

Runs the calculate_total function and returns a JSON FORMAT output as follows:
{"result": 240, "fn": "calculate_total", "next": "PAUSE"}

# 2. get_product_price:

This uses the get_product_price function and passes in the value of the product
e.g. get_product_price('bike')

This uses the get_product_price with a product = 'bike', finds the price of the bike and then returns a JSON FORMAT output as follows:
{"result": 200, "fn": "get_product_price", "next": "PAUSE"}

Here is an example session:

User Question: What is total cost of a bike including VAT?

AI Response: THOUGHT: I need to find the cost of a bike|ACTION|get_product_price|bike

You will be called again with the result of get_product_price as the OBSERVATION and will have OBSERVATION|200 sent as another LLM query along with previous messages.

Then the next message will be:

THOUGHT: I need to calculate the total including the VAT|ACTION|calculate_total|200

The result wil be passed as another query as OBSERVATION|240 along with previous messages.

If you have the ANSWER, output it as the ANSWER in this format:

ANSWER|The price of the bike including VAT is 240

"""

In [6]:
def calculate_total(amount) -> float:
    return int(amount * 1.2)


def get_product_price(product):
    if product == "bike":
        return 100
    if product == "tv":
        return 200
    if product == "laptop":
        return 300
    return None

In [7]:
agent = PlanningAgent(client=client, system=system_prompt)

In [8]:
result = agent("What is cost of a laptop including the VAT?")
# result = agent("What is cost of a bike including the VAT?")
# result = agent("What is cost of a tv including the VAT?")
print(result)

THOUGHT: I need to find the cost of a laptop|ACTION|get_product_price|laptop


We pass the result to the next call, having extracted the function name and arguments from the output.


In [9]:
next = result.split("|")
next_function = next[2]
next_arg = next[3]
print(f"FUNCTION - {next_function} |  ARGUMENT - {next_arg}")

FUNCTION - get_product_price |  ARGUMENT - laptop


In [10]:
# We manually run the tool - in the .py file example we do this programatically.
# This is for illustrative purposes.

if next_function == "get_product_price":
    result = get_product_price(next_arg)
    print(result)

300


In [11]:
next_prompt = f"Observation|{result}"
next_prompt

'Observation|300'

In [12]:
result = agent(next_prompt)
print(result)

THOUGHT: I need to calculate the total including the VAT|ACTION|calculate_total|300


In [13]:
next = result.split("|")
next_function = next[2]
next_arg = int(next[3])
print(f"FUNCTION - {next_function} |  ARGUMENT - {next_arg}")

FUNCTION - calculate_total |  ARGUMENT - 300


In [14]:
# We manually run the tool - in the .py file example we do this programatically.
# This is for illustrative purposes.

if next_function == "calculate_total":
    result = calculate_total(next_arg)
    print(result)

360


In [15]:
# We now create a new user prompt with the result and in the format specified in the prompt engineered system prompt
next_prompt = f"OBSERVATION|{result}"
next_prompt

'OBSERVATION|360'

In [16]:
# We now pass the OBSERVATION and RESULT as a new user query that is appended to all the previous messages.
result = agent(next_prompt)
print(result)

ANSWER|The price of the laptop including VAT is 360
